This function computes closed null geodesics of $ C_{t_0}^{t_N}(\mathbf{x})-\lambda^2 \mathbf{I} $, where $ C_{t_0}^{t_N}(\mathbf{x}, t) = \begin{pmatrix} C^{11} && C^{12} \\ C^{12} && C^{22} \end{pmatrix} $ is the (symmetric) Cauchy Green strain tensor.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X | array (Ny, Nx) | X-meshgrid|
| Y | array (Ny, Nx) | Y-meshgrid|
| lam | float | $ \lambda $|
| interp_phi_prime | interpolant | $ \dot{\phi} $|
| d_threshold | float | threshold distance between start point and return point of the closed curve |
| defined domain | boolean array (Ny, Nx) | meshgrid which denotes the domain where the velocity field is defined|
| interp_DOE | interpolant | $ \sin(2\phi)[C^{22}(\mathbf{x})-C^{11}(\mathbf{x})]+2\cos(2\phi)C^{12}(\mathbf{x}) $ |
| solODE_closed_curve | list | list containing the closed solution curves ($ \mathbf{x}, \phi $) |
| x0lam | list | list containing x-coordinates of initial conditions $ x_0(\lambda) $ |
| y0lam | list | list containing y-coordinates of initial conditions $ y_0(\lambda) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

In [2]:
# Import numpy
import numpy as np

# function computing initial conditions (depending on \lambda)
from ipynb.fs.defs.init_level_set import _init_level_set

# find closed curve
from ipynb.fs.defs.closed_curve import closed_curve

# RK4 integrator
from ipynb.fs.defs.RK4_integration import RK4_integration

In [3]:
def closed_null_geodesics(X, Y, lam, interp_phi_prime, d_threshold, C11, defined_domain, interp_DOE):
    
    # domain where the rate of strain field is defined
    defined_domain = np.isfinite(C11).astype(int)
    
    # compute initial conditions
    x0lam, y0lam, phi0lam = _init_level_set(X, Y, C11, lam)
    
    # define integration domain of dummy variable
    s = [0, 12]
    
    # define resolution of trajectories
    s_eval = np.linspace(s[0], s[1], 3000)
    
    ds = s_eval[1]-s_eval[0]
    
    # number of initial conditions
    len_x0lam = len(x0lam)
    
    solODE = np.zeros((len(s_eval), 3, len_x0lam))
    
    # initial conditions
    solODE[0, :, :] = np.array([x0lam, y0lam, phi0lam])
    
    for i in range(len(s_eval)-1):
        
        solODE[i+1, :, :] = RK4_integration(solODE[i, :, :], ds, interp_phi_prime)
    
    solODE_closed_curves = []
    
    # iterate over all initial conditions [x0lam, y0lam, phi0lam]
    for j in range(len_x0lam):
        
        # store x, y, phi
        x = solODE[::5,0, j]
        y = solODE[::5,1, j]
        phi = solODE[::5,2, j]
    
        # Check if curve is closed after completing one full cycle and find curve with minimum re-intersection distance.
        x_closed, y_closed, phi_closed = closed_curve(x, y, phi, d_threshold, X, Y, interp_DOE, defined_domain)
        
        # store solutions
        solODE_closed_curves.append([x_closed, y_closed, phi_closed])
    
    return solODE_closed_curves, [x0lam, y0lam]